In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
"""
For Natural Languages processing, nltk Library is used in python
nltk stands for natural language toolkit
As, We need to Tokenize the complete statement from corpus and from user entered string
for that, we need nltk.stem - stemming
and import wordNetLemmatizer
"""

import json
"""
Our data is present in json file
So, we are using json library
"""

import pickle
"""
As, I have mentioned above our data is present in json file.
We need to convert this data into byte stream thats why we are using pickle 
"""

import numpy as np
"""
Used to store large data, Majorly used in machine learning
"""

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
"""
For giving brain to our bot, we are using Sequential model from keras
We are using Dense, Activation and dropout layers 
Dense - layer which is deply connected with its preceding layers
Activation - 
Dropout - Helps to prevent neurons to set synchronous weights
SGD - Gardient descent - Find error and then increase accuracy of our bot
"""

import random
# To generate random responses


words=[]
classes = []
documents = []
ignore_words = ['?', '!']
"""
Lists:
To store words, classes and document
? and ! are ignored words, if enetred with question
"""

data_file = open('data.json').read()
intents = json.loads(data_file)

print(data_file)
print(intents)



{"intents": [
    {"tag": "greeting",
    "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
    "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
    "context": [""]
    },
    {"tag": "goodbye",
    "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
    "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
    "context": [""]
    },
    {"tag": "thanks",
    "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
    "responses": ["Happy to help!", "Any time!", "My pleasure"],
    "context": [""]
    },
    {"tag": "noanswer",
    "patterns": [],
    "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
    "context": [""]
    },
    {"tag": "options",
    "patterns": ["How you could help me?", "What you can do?", "What help you provide?", "H

In [13]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        print(w)
        words.extend(w)
    
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


['Hi', 'there']
['How', 'are', 'you']
['Is', 'anyone', 'there', '?']
['Hey']
['Hola']
['Hello']
['Good', 'day']
['Bye']
['See', 'you', 'later']
['Goodbye']
['Nice', 'chatting', 'to', 'you', ',', 'bye']
['Till', 'next', 'time']
['Thanks']
['Thank', 'you']
['That', "'s", 'helpful']
['Awesome', ',', 'thanks']
['Thanks', 'for', 'helping', 'me']
['How', 'you', 'could', 'help', 'me', '?']
['What', 'you', 'can', 'do', '?']
['What', 'help', 'you', 'provide', '?']
['How', 'you', 'can', 'be', 'helpful', '?']
['What', 'support', 'is', 'offered']
['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?']
['Open', 'adverse', 'drugs', 'module']
['Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior']
['List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction']
['Which', 'drugs', 'dont', 'have', 'adverse', 'reaction', '?']
['Open', 'blood', 'pressure', 'module']
['Task', 'related', 'to', 'blood', 'pressure']
['Blood', 'pressure', 'data', 'entry']
['I', 'wa

In [18]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print(words)

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
print(documents)

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
376 documents
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting

In [20]:
output_empty = [0] * len(classes)
print(output_empty)

[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
# create our training data
training = []
# create an empty list for our output
output_empty = [0] * len(classes)
print(output_empty)
print('\n')
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print(pattern_words)
    print('\n')
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

[0, 0, 0, 0, 0, 0, 0, 0, 0]


['Hi', 'there']


['How', 'are', 'you']


['Is', 'anyone', 'there', '?']


['Hey']


['Hola']


['Hello']


['Good', 'day']


['Bye']


['See', 'you', 'later']


['Goodbye']


['Nice', 'chatting', 'to', 'you', ',', 'bye']


['Till', 'next', 'time']


['Thanks']


['Thank', 'you']


['That', "'s", 'helpful']


['Awesome', ',', 'thanks']


['Thanks', 'for', 'helping', 'me']


['How', 'you', 'could', 'help', 'me', '?']


['What', 'you', 'can', 'do', '?']


['What', 'help', 'you', 'provide', '?']


['How', 'you', 'can', 'be', 'helpful', '?']


['What', 'support', 'is', 'offered']


['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?']


['Open', 'adverse', 'drugs', 'module']


['Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior']


['List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction']


['Which', 'drugs', 'dont', 'have', 'adverse', 'reaction', '?']


['Open', 'blood', 'pressure', 'module']


['Task', 

C:\Users\uilya\AppData\Local\Temp\ipykernel_14296\773357846.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
